In [26]:
import math

#https://edwilliams.org/gccalc.htm

sk = 48.150245, 17.066950
sk2 = 48.340323, 17.696664

pt = 38.690840, -8.177875

#wgs84
#hdg 1-2 108.02042013
#hdg 2-1 305.58084939
#dist 2284.02804834

#FAI
#hdg 1-2 108.10249438
#hdg 2-1 305.66226711
#dist 2279.59044552

#drop old
#2296.3710722089704

from geopy.distance import great_circle
from geopy.distance import vincenty


GPS_MULT = 10000000.0
M_PI = 3.14159265358979323846
FAI_EARTH_RADIUS = 6371

def distance_old(lat1, lon1, lat2, lon2):

    lat1 *= GPS_MULT
    lon1 *= GPS_MULT
    lat2 *= GPS_MULT
    lon2 *= GPS_MULT
    
    # Compute the average lat of lat1 and lat2 to get the width of a
    # 1 degree cell at that position of the earth:
    lat = (lat1 + lat2) / 2 / GPS_MULT * (M_PI / 180.0);

    # 111.3 km (in cm) is the width of 1 degree
    # disp.get_cos() is not usable here, because we deal with fractions of a degree.
    dx = math.cos(lat) * 111.3 * abs(lon1 - lon2) / GPS_MULT;
    dy = 1.0      * 111.3 * abs(lat1 - lat2) / GPS_MULT;

    return math.sqrt(dx * dx + dy * dy);


def distance_new(lat1, lon1, lat2, lon2):
    lat1 *= M_PI / 180.0
    lat2 *= M_PI / 180.0
    lon1 *= M_PI / 180.0
    lon2 *= M_PI / 180.0
    
    dlat = (lat2 - lat1) / 2
    dlon = (lon2 - lon1) / 2
    print(" dlon", dlon)
    print(" dlat", dlat)

    a = math.sin(dlat) ** 2 + ((math.sin(dlon) ** 2) * math.cos(lat1) * math.cos(lat2))
    print(" q", a)
    
    return 2 * FAI_EARTH_RADIUS * math.asin(math.sqrt(a));


class CheapRuler(object):
    
    def init(self, latitude):
           
        print( " latitude", latitude)
        cos = math.cos(latitude * M_PI / 180.);
        cos2 = 2. * cos * cos - 1.;
        cos3 = 2. * cos * cos2 - cos;
        cos4 = 2. * cos * cos3 - cos2;
        cos5 = 2. * cos * cos4 - cos3;

        # multipliers for converting longitude and latitude
        # degrees into distance (http://1.usa.gov/1Wb1bv7)
        self.kx = (111.41513 * cos - 0.09455 * cos3 + 0.00012 * cos5);
        self.ky = (111.13209 - 0.56605 * cos2 + 0.0012 * cos4);
        print(" kx", self.kx);
        print(" ky", self.ky);
    
    def dist(self, lat1, lon1, lat2, lon2):
        dx = (lon1 - lon2) * self.kx;
        dy = (lat1 - lat2) * self.ky;

        print(" d_lon", dx);
        print(" d_lon", dy);
        return math.sqrt(dx * dx + dy * dy) ;     
    
    def bearing(self, lat1, lon1, lat2, lon2):
        dx = (lon1 - lon2) * self.kx;
        dy = (lat1 - lat2) * self.ky;

        if (dx  == 0 and dy == 0):
            return 0

        value = math.atan2(dx, dy) * 180. / M_PI;

        if (value > 180.):
            value -= 360.
        

        return value;
        

lat1, lon1 = sk
lat2, lon2 = sk2

pt = sk2

vin = vincenty(sk, pt).kilometers
old = distance_old(lat1, lon1, lat2, lon2)
new = distance_new(lat1, lon1, lat2, lon2)
fai = great_circle(sk, pt, radius = FAI_EARTH_RADIUS).kilometers

cr = CheapRuler()
cr.init((lat1 + lat2) / 2)
cr = cr.dist(lat1, lon1, lat2, lon2)

print("old\t\t", old)
print("vincenty\t", vin)
print("cheap\t\t", cr, abs(cr - vin))
print("great_circle\t", fai)
print("new FAI\t\t", new, abs(new - fai))


 dlon 0.00549529132295179
 dlat 0.0016587434678029256
 q 1.614352433116328e-05
 latitude 48.245284
 kx 74.27327610653239
 ky 111.19490676154619
 d_lon -46.77092179014893
 d_lon -21.13570548742115
old		 51.24481546355443
vincenty	 51.32327170545291
cheap		 51.324820229117535 0.0015485236646242129
great_circle	 51.19622576615805
new FAI		 51.19622576615808 2.842170943040401e-14


/home/horinek/.local/lib/python3.6/site-packages/ipykernel_launcher.py:114: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.


In [38]:
def get_dist(lat1, lon1, lat2, lon2, f, a, d):
    d /= 1000
    
    lat1 /= GPS_MULT
    lat2 /= GPS_MULT
    lon1 /= GPS_MULT
    lon2 /= GPS_MULT
    
    p1 = (lat1, lon1)
    p2 = (lat2, lon2)
    
    vin = vincenty(p1, p2).kilometers
    fai = great_circle(p1, p2, radius = FAI_EARTH_RADIUS).kilometers    

    if (f == 0):
        cr = CheapRuler()
        cr.init((lat1 + lat2) / 2)
        cr = cr.dist(lat1, lon1, lat2, lon2)
        
        print("vincenty\t%6.4f" % vin)
        print("cheap\t\t%6.4f\t%6.4f" % (cr, (cr - vin)))
        print("drop\t\t%6.4f\t%6.4f" % (d, (d - vin)))
    
    else:
        new = distance_new(lat1, lon1, lat2, lon2)
        
        print("great_circle\t%6.4f" % fai)
        print("new FAI\t\t%6.4f\t%6.4f" % (new, (new - fai)))
        print("drop\t\t%6.4f\t%6.4f" % (d, (d - fai)))

    print()
    

#d_lon=0.0434845020
#d_lat=0.1503360000
lat1=484188640
lon1=179139488
lat2=485692000
lon2=179574333
f=1
#d_lon=0.0003794739
#d_lat=0.0013119291
#q=0.0000017844
a=16
d=17020
get_dist(lat1, lon1, lat2, lon2, f, a, d)


 dlon 0.00037947384929173933
 dlat 0.0013119290921391191
 q 1.7843971760159335e-06
great_circle	17.0209
new FAI		17.0209	0.0000
drop		17.0200	-0.0009



/home/horinek/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  if sys.path[0] == '':
